## Counting Organizations

This application will read the mailbox data (mbox.txt) and count the number of email messages per organization (i.e. domain name of the email address) using a database with the following schema to maintain the counts.

CREATE TABLE Counts (org TEXT, count INTEGER)

When you have run the program on mbox.txt upload the resulting database file above for grading.

If you run the program multiple times in testing or with dfferent files, make sure to empty out the data before each run.

You can use this code as a starting point for your application: http://www.py4e.com/code3/emaildb.py.

The data file for this application is the same as in previous assignments: http://www.py4e.com/code3/mbox.txt.

Because the sample code is using an UPDATE statement and committing the results to the database as each record is read in the loop, it might take as long as a few minutes to process all the data. The commit insists on completely writing all the data to disk every time it is called.

The program can be speeded up greatly by moving the commit operation outside of the loop. In any database program, there is a balance between the number of operations you execute between commits and the importance of not losing the results of operations that have not yet been committed. 

In [10]:
import sqlite3
#Connecting to the database
#Since there is no database named emaildb.sqlite, it will create one
conn = sqlite3.connect('emaildb.sqlite')
#This is like the handle in file handling
cur = conn.cursor()

#Drop the table named Counts if already exists
#So that our sql statements won't turn out useless since there could already be such table
cur.execute('DROP TABLE IF EXISTS Counts')

cur.execute('''
    CREATE TABLE Counts (
        org TEXT,
        count INTEGER
    )
''')

#This is the part where we read from the text file
#And split the lines which start with "From: "
#And check the email's domain, which will be the name of the organization
fname = input('Enter the file name: ')
if(len(fname)<1): fname = 'mbox.txt'

handle = open(fname)
for line in handle:
    if not line.startswith('From: '): 
        continue
    pieces = line.split()[1]
    org = pieces.split('@')[1]
    
    cur.execute('SELECT count FROM Counts WHERE org = ?', (org,))
    row = cur.fetchone()
    if row is None:
        cur.execute('''
            INSERT INTO Counts (org, count)
            VALUES (?, 1)
        ''', (org,))
    else:
        cur.execute('''
            UPDATE Counts SET count = count + 1 WHERE org = ?
        ''', (org,))

#The commit function actually writes the data into the disk/drive
#Before commit, they are only on memory
conn.commit()

query = 'SELECT * FROM Counts'

#row is a tuple
#so row[0] and row[1] will return the org and count resp
for row in cur.execute(query):
    print(row[0], row[1])
    
cur.close()

Enter the file name: 
uct.ac.za 96
media.berkeley.edu 56
umich.edu 491
iupui.edu 536
caret.cam.ac.uk 157
gmail.com 25
indiana.edu 178
et.gatech.edu 17
vt.edu 110
lancaster.ac.uk 14
ucdavis.edu 1
ufp.pt 28
txstate.edu 17
stanford.edu 12
whitman.edu 17
rsmart.com 8
fhda.edu 1
bu.edu 14
unicon.net 9
loi.nl 9
utoronto.ca 1
